In [ ]:
# training json 파일 포매팅
import os
import json
import codecs


def process_files_with_preserved_structure(
    input_folder, output_base_folder, overwrite=True
):
    """
    입력 폴더 구조를 보존하며 JSON 파일을 처리하여 결과를 저장.
    덮어쓰기가 활성화된 경우 이미 저장된 파일도 덮어씁니다.
    유효하지 않은 JSON 파일은 건너뛰고 출력합니다.

    :param input_folder: 입력 파일들이 위치한 부모 폴더 경로
    :param output_base_folder: 결과 파일을 저장할 최상위 폴더 경로
    :param overwrite: 이미 존재하는 파일을 덮어쓸지 여부 (기본값: True)
    """
    skipped_files = []  # 유효하지 않은 JSON 파일 목록

    for root, dirs, files in os.walk(input_folder):
        for file_name in files:
            if file_name.endswith(".json"):  # JSON 파일만 처리
                input_file_path = os.path.join(root, file_name)

                # 출력 폴더 구조 생성
                relative_path = os.path.relpath(root, input_folder)
                output_folder = os.path.join(output_base_folder, relative_path)
                os.makedirs(output_folder, exist_ok=True)

                output_file_path = os.path.join(output_folder, file_name)

                # 덮어쓰기 여부 확인
                if not overwrite and os.path.exists(output_file_path):
                    print(f"이미 존재하는 파일, 건너뜁니다: {output_file_path}")
                    continue

                try:
                    # JSON 파일 로드
                    with codecs.open(
                        input_file_path, "r", encoding="utf-8-sig"
                    ) as file:
                        data = json.load(file)

                    # OCR_info에서 필요한 항목 추출 및 순서 변경
                    filtered_ocr_info = []
                    for ocr_item in data.get("OCR_info", []):
                        filtered_ocr_info.append(
                            {
                                "question_text": ocr_item.get(
                                    "question_text"
                                ),  # question_text 먼저 추가
                                "figure_text": ocr_item.get(
                                    "figure_text"
                                ),  # figure_text 다음
                                "question_bbox": [
                                    bbox
                                    for bbox in ocr_item.get("question_bbox", [])
                                    if bbox.get("type")
                                    in ["line", "image"]  # 'line'과 'image' 타입만 추출
                                ],
                            }
                        )

                    # 최종 데이터 구성
                    output_data = {"OCR_info": filtered_ocr_info}

                    # 결과 JSON 파일 저장
                    with open(output_file_path, "w", encoding="utf-8") as output_file:
                        json.dump(
                            output_data, output_file, ensure_ascii=False, indent=4
                        )

                    print(f"JSON 파일이 처리되어 저장되었습니다: {output_file_path}")

                except (json.JSONDecodeError, FileNotFoundError) as e:
                    print(
                        f"유효하지 않은 JSON 파일을 건너뜁니다: {input_file_path}, 에러: {e}"
                    )
                    skipped_files.append({"file": input_file_path, "error": str(e)})

    # 유효하지 않은 파일 출력
    if skipped_files:
        print("\n유효하지 않아 건너뛴 JSON 파일 목록:")
        for skipped in skipped_files:
            print(f" - 파일: {skipped['file']}, 에러: {skipped['error']}")


# 실행
input_dir = "./training/y"  # 입력 JSON 파일들이 위치한 폴더
output_base_dir = "./training/formatted_y"  # 출력 파일을 저장할 최상위 폴더
process_files_with_preserved_structure(input_dir, output_base_dir, overwrite=True)

JSON 파일이 처리되어 저장되었습니다: ./training/formatted_y\elementary3\P3_1_01_21114_49495.json
JSON 파일이 처리되어 저장되었습니다: ./training/formatted_y\elementary3\P3_1_01_21114_49496.json
JSON 파일이 처리되어 저장되었습니다: ./training/formatted_y\elementary3\P3_1_01_21114_49497.json
JSON 파일이 처리되어 저장되었습니다: ./training/formatted_y\elementary3\P3_1_01_21114_49506.json
JSON 파일이 처리되어 저장되었습니다: ./training/formatted_y\elementary3\P3_1_01_21116_49499.json
JSON 파일이 처리되어 저장되었습니다: ./training/formatted_y\elementary3\P3_1_01_21116_49500.json
JSON 파일이 처리되어 저장되었습니다: ./training/formatted_y\elementary3\P3_1_01_21116_49501.json
JSON 파일이 처리되어 저장되었습니다: ./training/formatted_y\elementary3\P3_1_01_21116_49502.json
JSON 파일이 처리되어 저장되었습니다: ./training/formatted_y\elementary3\P3_1_01_21116_49503.json
JSON 파일이 처리되어 저장되었습니다: ./training/formatted_y\elementary3\P3_1_01_21116_49504.json
JSON 파일이 처리되어 저장되었습니다: ./training/formatted_y\elementary3\P3_1_01_21116_49505.json
JSON 파일이 처리되어 저장되었습니다: ./training/formatted_y\elementary3\P3_1_01_21160_49964.json
JSON

In [ ]:
# validation json 파일 포매팅
import os
import json
import codecs


def process_files_with_preserved_structure(
    input_folder, output_base_folder, overwrite=True
):
    """
    입력 폴더 구조를 보존하며 JSON 파일을 처리하여 결과를 저장.
    덮어쓰기가 활성화된 경우 이미 저장된 파일도 덮어씁니다.
    유효하지 않은 JSON 파일은 건너뛰고 출력합니다.

    :param input_folder: 입력 파일들이 위치한 부모 폴더 경로
    :param output_base_folder: 결과 파일을 저장할 최상위 폴더 경로
    :param overwrite: 이미 존재하는 파일을 덮어쓸지 여부 (기본값: True)
    """
    skipped_files = []  # 유효하지 않은 JSON 파일 목록

    for root, dirs, files in os.walk(input_folder):
        for file_name in files:
            if file_name.endswith(".json"):  # JSON 파일만 처리
                input_file_path = os.path.join(root, file_name)

                # 출력 폴더 구조 생성
                relative_path = os.path.relpath(root, input_folder)
                output_folder = os.path.join(output_base_folder, relative_path)
                os.makedirs(output_folder, exist_ok=True)

                output_file_path = os.path.join(output_folder, file_name)

                # 덮어쓰기 여부 확인
                if not overwrite and os.path.exists(output_file_path):
                    print(f"이미 존재하는 파일, 건너뜁니다: {output_file_path}")
                    continue

                try:
                    # JSON 파일 로드
                    with codecs.open(
                        input_file_path, "r", encoding="utf-8-sig"
                    ) as file:
                        data = json.load(file)

                    # OCR_info에서 필요한 항목 추출 및 순서 변경
                    filtered_ocr_info = []
                    for ocr_item in data.get("OCR_info", []):
                        filtered_ocr_info.append(
                            {
                                "question_text": ocr_item.get(
                                    "question_text"
                                ),  # question_text 먼저 추가
                                "figure_text": ocr_item.get(
                                    "figure_text"
                                ),  # figure_text 다음
                                "question_bbox": [
                                    bbox
                                    for bbox in ocr_item.get("question_bbox", [])
                                    if bbox.get("type")
                                    in ["line", "image"]  # 'line'과 'image' 타입만 추출
                                ],
                            }
                        )

                    # 최종 데이터 구성
                    output_data = {"OCR_info": filtered_ocr_info}

                    # 결과 JSON 파일 저장
                    with open(output_file_path, "w", encoding="utf-8") as output_file:
                        json.dump(
                            output_data, output_file, ensure_ascii=False, indent=4
                        )

                    print(f"JSON 파일이 처리되어 저장되었습니다: {output_file_path}")

                except (json.JSONDecodeError, FileNotFoundError) as e:
                    print(
                        f"유효하지 않은 JSON 파일을 건너뜁니다: {input_file_path}, 에러: {e}"
                    )
                    skipped_files.append({"file": input_file_path, "error": str(e)})

    # 유효하지 않은 파일 출력
    if skipped_files:
        print("\n유효하지 않아 건너뛴 JSON 파일 목록:")
        for skipped in skipped_files:
            print(f" - 파일: {skipped['file']}, 에러: {skipped['error']}")


# 실행
input_dir = "./validation/y"  # 입력 JSON 파일들이 위치한 폴더
output_base_dir = "./validation/formatted_y"  # 출력 파일을 저장할 최상위 폴더
process_files_with_preserved_structure(input_dir, output_base_dir, overwrite=True)

JSON 파일이 처리되어 저장되었습니다: ./validation/formatted_y\elementary3\P3_1_01_21114_49498.json
JSON 파일이 처리되어 저장되었습니다: ./validation/formatted_y\elementary3\P3_1_01_21169_50033.json
JSON 파일이 처리되어 저장되었습니다: ./validation/formatted_y\elementary3\P3_1_01_21180_50074.json
JSON 파일이 처리되어 저장되었습니다: ./validation/formatted_y\elementary3\P3_1_01_21186_50114.json
JSON 파일이 처리되어 저장되었습니다: ./validation/formatted_y\elementary3\P3_1_01_21187_50121.json
JSON 파일이 처리되어 저장되었습니다: ./validation/formatted_y\elementary3\P3_1_01_21187_50123.json
JSON 파일이 처리되어 저장되었습니다: ./validation/formatted_y\elementary3\P3_1_01_21193_50168.json
JSON 파일이 처리되어 저장되었습니다: ./validation/formatted_y\elementary3\P3_1_01_21236_50304.json
JSON 파일이 처리되어 저장되었습니다: ./validation/formatted_y\elementary3\P3_1_01_21244_50356.json
JSON 파일이 처리되어 저장되었습니다: ./validation/formatted_y\elementary3\P3_1_01_21250_50397.json
JSON 파일이 처리되어 저장되었습니다: ./validation/formatted_y\elementary3\P3_1_01_21258_50435.json
JSON 파일이 처리되어 저장되었습니다: ./validation/formatted_y\elementary3\P3_1_